In [1]:
#Set up DRWatson
cd("/home/jm2386/Active_Lattice/")
using DrWatson
@quickactivate "Active_Lattice"
# Load relevant fuctions
include("/home/jm2386/Active_Lattice/src/pm_pde_functions.jl")
include("/home/jm2386/Active_Lattice/src/Hetrocline.jl")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `/mhome/damtp/s/jm2386/Active_Lattice/Project.toml`
  No Changes to `/mhome/damtp/s/jm2386/Active_Lattice/Manifest.toml`


fake_spin_boundary (generic function with 1 method)

In [2]:
# set parameters
ϕa = 0.6
ϕp = 0.1
Pe = 7.5
#
ρ = ϕa+ϕp
χ = ϕa/ρ
pert = "pm_lin"
    T  = 0.1
    δ  = 1e-4
    save_interval = 0.0001
    Dx = 1. 
    Dθ = 10000.0
    Nx = 512
    Nθ = 2
name = "pm_bin_test_9_pde_δ=$(δ)_l=$(1/sqrt(Dθ))"
#
param = pde_param_pm(; name = name, 
                                ρ = ρ, Pe = Pe, χ = χ, T = T, 
                                Dθ = Dθ, δt = 1e-5, Nx = Nx, Nθ = Nθ, 
                                save_interval = save_interval, max_steps = 1e7,
                                pert = pert, δ = δ,
);#compute bindoal
Pe = 10.0
initial_Δ = 1e-4;
max_iter = 40;
tol = 1e-3;
atol = 1e-12;
rho_max = (1-10e-20);
γ = (1-ϕa)/(1-ρ);

find_sol, lower_limits, upper_limits = colapse_sol_interval(;Pe = Pe, γ = γ, rho_max = rho_max, initial_Δ = initial_Δ, max_iter = max_iter, tol = tol, atol = atol);
ϕg = lower_limits[1]
ϕl = upper_limits[1]
ϕg, ϕl 

(0.35985638771544043, 0.9693916139039973)

In [3]:
#compute velocitites
ϵ = 1e-10
Δx = 0.0001
Δy = 0.0001

x = collect((ϕg-0.1+ϵ):Δx:(0.9999))
y = collect((-0.999):Δy:0.999)

xx = [x̃ for x̃ ∈ x, ỹ ∈ y]
    yy = [ỹ for x̃ ∈ x, ỹ ∈ y]
    Nx = length(x)
    Ny = length(y)
    v = zeros(Nx,Ny,2)
    for i in 1:Nx, j in 1:Ny
        ρ = x[i]
        m = y[j]
        v[i,j,1] = Pe*(1-ρ)*m
        v[i,j,2] = -Pe*m^2 + Pe*( (1-γ*(1-ρ) )*self_diff(ρ) -( 1-γ*(1- ϕg) )*self_diff(ϕg) )/self_diff(ρ) -(2/Pe)*log( (1-ρ)/(1-ϕg) )/self_diff(ρ)
    end

In [4]:
#plot fig
PyPlot.close("all")
fig, ax = plt.subplots(1, 1, figsize=(10,10))
    max_value = 1
    absmag  = min.(sqrt.(v[:,:,1].^2+v[:,:,2].^2), max_value)
    colmap = PyPlot.plt.cm.viridis
    streamlines = ax.streamplot(xx', yy', v[:,:,1]', v[:,:,2]', color = absmag', cmap = colmap, density = 4)#2.5
    norm1 = matplotlib.colors.Normalize(vmin=0., vmax= max_value);
    fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm1, cmap = colmap), ax = ax, fraction = 0.0455)
pertubation = [ϕg+ϵ 0.0]
ax.streamplot(xx', yy', v[:,:,1]', v[:,:,2]', color="red", start_points= pertubation)
# fig options
rc("text", usetex=true)
    #ax.yaxis.set_ticks(ytic)
    axlim = [0.2, 1.0, -1.0, 1.0]
    ax.xaxis.set_tick_params(labelsize=15)
    ax.xaxis.tick_bottom()
    ax.yaxis.set_tick_params(labelsize=15)
    ax.axis(axlim)
    #ax.set_title(L"\Re{ \lambda_n^\mathrm{max}} = 0",fontsize=20)
    ax.set_xlabel(L"\rho",fontsize=20)
    ax.set_ylabel(L"m", fontsize=20)
    #ax.legend(loc = "upper left", fontsize=20)
    #ax.set_aspect(0.25*Δρ/ΔPe)
    #title = latexstring("\$ \\ell = $(round(1/sqrt(Dθ); digits = 2)), \\chi = $(χ) \$")
    #ax.set_title(title,fontsize=20)
    fig.tight_layout()
display(fig)

In [ ]:
#save fig
name = "Streamplot_Pe=$(Pe)"
    pathname = "/store/DAMTP/jm2386/Active_Lattice/plots/active_passive_1d/$(name)";
    mkpath(pathname)
    filename = "/store/DAMTP/jm2386/Active_Lattice/plots/active_passive_1d/$(name)/Streamplot.pdf";
    PyPlot.savefig(filename,dpi = 100, format = "pdf") #bbox_extra_artists=( ldg,)
##